In [1]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import Range1d
from bokeh.io import export_png
output_notebook()

import numpy as np

from voltagebudget import util
from voltagebudget import neurons
from voltagebudget import plc
from voltagebudget.util import locate_firsts
from voltagebudget.util import locate_peaks
from voltagebudget.util import filter_voltages

# -
def create_traces(ts_x, changes, fn):
    d_variances = []
    errors = []
    for c in changes:
        initial, target, obs, ts_opt = fn(ts_x, c)

        d_v = initial - obs
        err = util.mae(ts_x, ts_opt)

        d_variances.append(d_v)
        errors.append(err)

    d_variances = np.asarray(d_variances)
    errors = np.asarray(errors)

    return d_variances, errors


Loading BokehJS ...

/Users/type/anaconda/envs/py2/lib/python2.7/site-packages/brian2/core/variables.py:174: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return np.issubdtype(np.bool, self.dtype)


In [2]:
changes = np.linspace(0.0, 0.99, 50)

t=0.5
stim_number=40
stim_onset=0.2
stim_offset=0.250
stim_rate=8
seed_stim=7525
time_step=1e-5

ns_x, ts_x = util.poisson_impulse(
        t,
        stim_onset,
        stim_offset - stim_onset,
        stim_rate,
        dt=time_step,
        n=stim_number,
        seed=seed_stim)

In [3]:
p = figure(plot_width=400, plot_height=200)
p.circle(ts_x, ns_x, color="grey", alpha=0.3)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Uniform

In [4]:
percent_change = 0.7

initial, target, obs, ts_opt = plc.uniform(ts_x, percent_change)

p = figure(plot_width=400, plot_height=200)
p.circle(ts_opt, ns_x, color="black")
p.circle(ts_x, ns_x, color="grey", alpha=0.3)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [5]:
# -
d_variances_u, errors_u = create_traces(ts_x, changes, plc.uniform)

p = figure(plot_width=400, plot_height=200)
p.line(x=d_variances_u*1e3, y=errors_u*1e3, color="black")
p.xaxis.axis_label = 'Delta Variance (ms)'
p.yaxis.axis_label = 'Error (ms)'
# p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Extreme deviants

## Max

In [6]:
percent_change = 0.4

initial, target, obs, ts_opt = plc.max_deviant(ts_x, percent_change)

p = figure(plot_width=400, plot_height=200)
p.circle(ts_opt, ns_x, color="black")
p.circle(ts_x, ns_x, color="grey", alpha=0.3)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

print(initial, target, obs)

(0.012855946336618894, 0.007713567801971336, 0.007712736460076256)


In [7]:
# -
d_variances_ma, errors_ma = create_traces(ts_x, changes, plc.max_deviant)

p = figure(plot_width=400, plot_height=200)
p.line(x=d_variances_ma*1e3, y=errors_ma*1e3, color="black")
p.xaxis.axis_label = 'Delta Variance (ms)'
p.yaxis.axis_label = 'Error (ms)'
# p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Compare traces

In [8]:
# -

p = figure(plot_width=400, plot_height=200)
p.line(x=d_variances_ma*1e3, y=errors_ma*1e3, color="black", legend="Max")
p.line(x=d_variances_u*1e3, y=errors_u*1e3, color="orange", legend="Unif")
p.xaxis.axis_label = 'Delta Variance (ms)'
p.yaxis.axis_label = 'Error (ms)'
# p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)